<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np

import spacy
import re

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import f1_score

In [2]:
df_tweets = pd.read_csv("/datasets/toxic_comments.csv")

In [3]:
# взяла одну треть, т.к. долго ждать (если из-за этого завернете - исправлюсь)
df_tweets = df_tweets.sample(50000).reset_index(drop=True)

In [4]:
df_tweets.head()

,Unnamed: 0,text,toxic
0,151015,"LGBT \n\nyou little fuck , are you a fag , tha...",1
1,2201,"""\n\n""""The Wiki Foundation stats, in an email ...",0
2,74441,I agree. Nazi is not a critical problem in São...,0
3,25958,Get Off My Talk Page U Troublemakers. Buzz Of...,0
4,25243,"""\n\nI'm giving you the opportunity to be unbl...",0


In [5]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  50000 non-null  int64 
 1   text        50000 non-null  object
 2   toxic       50000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 1.1+ MB


In [6]:
# посмотрим на пропуски
df_tweets.isna().mean()

Unnamed: 0    0.0
text          0.0
toxic         0.0
dtype: float64

In [7]:
# посмотрим на наличие дубликатов
df_tweets.duplicated().sum()

0

In [8]:
# удалим ненужный столбец с номерацией
df_tweets = df_tweets.drop(['Unnamed: 0'], axis=1)

In [9]:
# проверим на сбалансированность классов
df_tweets['toxic'].value_counts()

0    44969
1     5031
Name: toxic, dtype: int64

In [10]:
#corpus = list(df_tweets['text'])

In [11]:
# напишем функцию lemmatize(text), которая на вход она принимает текст и возвращает лемматизированную строку
nlp = spacy.load('en_core_web_sm')
def lemmatize(text):
    lemm_list = nlp(text)
    lemm_list2 = []
    # удалим стоп-слова
    for token in lemm_list:
        if not token.is_stop:
            lemm_list2.append(token)
    # соединим токены через пробелы
    lemm_text = " ".join([token.lemma_ for token in lemm_list2])    
    return lemm_text

In [12]:
# напишем функцию clear_text(text), которая оставит в тексте только кириллические символы и пробелы
def clear_text(text):
    text = text.lower()
    reg = re.sub(r'[^a-z]', ' ', text).split()
    reg = " ".join(reg)
    return reg

In [13]:
# проверим правильность выполнения функций по лемматизации и очистке
sentence = "The striped bats are ^ hanging on their6 feet for best"
lemmatize(clear_text(sentence))

'stripe bat hang foot good'

In [14]:
# применим функцию очистки к каждой строке столбца 'text'
df_tweets['clean_text'] = df_tweets['text'].apply(clear_text)

In [15]:
# применим функцию лемматизации к каждой строке столбца 'clean_text'
df_tweets['lem_text'] = df_tweets['clean_text'].apply(lemmatize)

In [16]:
df_tweets.head()

,text,toxic,clean_text,lem_text
0,"LGBT \n\nyou little fuck , are you a fag , tha...",1,lgbt you little fuck are you a fag that piece ...,lgbt little fuck fag piece shit page make fuck...
1,"""\n\n""""The Wiki Foundation stats, in an email ...",0,the wiki foundation stats in an email forwarde...,wiki foundation stat email forward rti uk chap...
2,I agree. Nazi is not a critical problem in São...,0,i agree nazi is not a critical problem in s o ...,agree nazi critical problem s o paulo place br...
3,Get Off My Talk Page U Troublemakers. Buzz Of...,0,get off my talk page u troublemakers buzz off ...,talk page u troublemaker buzz lose stalker
4,"""\n\nI'm giving you the opportunity to be unbl...",0,i m giving you the opportunity to be unblocked...,m give opportunity unblock note user talk gran...


In [17]:
# избавимся от неочищенных столбцов
df_tweets = df_tweets.drop(['text', 'lem_text'], axis=1)

In [18]:
df_tweets.head()

,toxic,clean_text
0,1,lgbt you little fuck are you a fag that piece ...
1,0,the wiki foundation stats in an email forwarde...
2,0,i agree nazi is not a critical problem in s o ...
3,0,get off my talk page u troublemakers buzz off ...
4,0,i m giving you the opportunity to be unblocked...


**В данных 159292 стрки, один столбец бинарной классификации и один с текстом. Столбец с текстом мы очистили от лишних символов и привели все слова к лемме, удалив короткие стоп-слова. Пропусков в данных нет, повторов нет. Будем решать задачу классификации обучения с учителем.**

## Обучение

In [19]:
# выделим признаки
features = df_tweets['clean_text']
target = df_tweets['toxic']

X_train, X_test, y_train, y_test = train_test_split(
    features,
    target,
    test_size=0.10,
    random_state=12345,
    stratify=target
)

In [20]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
y_test.shape

(45000,)
(5000,)
(45000,)


(5000,)

**Модель логистической регрессии LogisticRegression**

In [21]:
# используем конвейер, чтобы обучение кодировщика при кросс-валидации производилось только на фолдах в состоянии обучения
lr_mpipe = make_pipeline(TfidfVectorizer(), LogisticRegression(class_weight='balanced', random_state=12345))
lr_mpipe.steps[1]

('logisticregression',
 LogisticRegression(class_weight='balanced', random_state=12345))

In [22]:
%%time
# используем поиск по сетке GridSearchCV для оптимизации гиперпараметров
params_lr = {'logisticregression__C':range(8, 12, 2)}

lr_gscv = GridSearchCV(
    lr_mpipe,
    params_lr,
    scoring='f1',
    cv=3,   # количество блоков для кросс-валидации
    verbose=3  # видеть время
)

lr_gscv.fit(X_train, y_train)

print(f'Лучша оценка логистической регрессии: {lr_gscv.best_score_}')

print(f'Лучшие гиперпараметры логистической регрессии: {lr_gscv.best_params_}')

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 1/3] END ........................logisticregression__C=8; total time=  33.7s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/3] END ........................logisticregression__C=8; total time=  33.4s
[CV 3/3] END ........................logisticregression__C=8; total time=  30.3s


...

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 2/3] END .......................logisticregression__C=10; total time=  32.9s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[CV 3/3] END .......................logisticregression__C=10; total time=  35.8s
Лучша оценка логистической регрессии: 0.7435118708764872
Лучшие гиперпараметры логистической регрессии: {'logisticregression__C': 10}
CPU times: user 1min 44s, sys: 2min 16s, total: 4min
Wall time: 4min 1s


/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


**Модель градиентного бустинга CatBoostClassifier**

In [23]:
# используем конвейер
cat_bc = make_pipeline(TfidfVectorizer(), CatBoostClassifier(random_state=12345, class_weights=(1, 9)))
cat_bc.steps[1]

('catboostclassifier', <catboost.core.CatBoostClassifier at 0x7f0be18d9b80>)

In [24]:
%%time
# используем поиск по сетке GridSearchCV
params_cat = {'catboostclassifier__iterations': [50, 100, 150]}

cat_gscv = GridSearchCV(
    cat_bc,
    params_cat,
    scoring='f1',
    cv=3,   # количество блоков для кросс-валидации
    verbose=3  # видеть время
)

cat_gscv.fit(X_train, y_train)

print(f'Лучша оценка логистической регрессии: {cat_gscv.best_score_}')

print(f'Лучшие гиперпараметры логистической регрессии: {cat_gscv.best_params_}')

Fitting 3 folds for each of 3 candidates, totalling 9 fits
Learning rate set to 0.5
0:	learn: 0.5669332	total: 1.17s	remaining: 57.3s
1:	learn: 0.5315449	total: 1.95s	remaining: 46.9s
2:	learn: 0.4991424	total: 2.76s	remaining: 43.3s
3:	learn: 0.4802013	total: 3.63s	remaining: 41.8s
4:	learn: 0.4567064	total: 4.41s	remaining: 39.7s
5:	learn: 0.4472011	total: 5.18s	remaining: 38s
6:	learn: 0.4317324	total: 5.97s	remaining: 36.7s
7:	learn: 0.4142416	total: 6.81s	remaining: 35.7s
8:	learn: 0.4063328	total: 7.57s	remaining: 34.5s
9:	learn: 0.4011521	total: 8.36s	remaining: 33.4s
10:	learn: 0.3962867	total: 9.19s	remaining: 32.6s
11:	learn: 0.3896097	total: 10s	remaining: 31.7s
12:	learn: 0.3799667	total: 10.8s	remaining: 30.8s
13:	learn: 0.3737029	total: 11.7s	remaining: 30s
14:	learn: 0.3669578	total: 12.5s	remaining: 29.2s
15:	learn: 0.3614546	total: 13.5s	remaining: 28.6s
16:	learn: 0.3566012	total: 14.3s	remaining: 27.7s
17:	learn: 0.3516564	total: 15.1s	remaining: 26.8s
18:	learn: 0.3

**Градиентный бустинг над решающими деревьями LGBMClassifier**

In [25]:
# используем конвейер
lgbm_cl = make_pipeline(TfidfVectorizer(), LGBMClassifier(random_state=12345))
lgbm_cl.steps[1]

('lgbmclassifier', LGBMClassifier(random_state=12345))

In [26]:
%%time
# используем поиск по сетке GridSearchCV
params_lgbm = {
    'lgbmclassifier__n_estimators':[300],
    'lgbmclassifier__max_depth':[20]
}

lgbm_gscv = GridSearchCV(
    lgbm_r,
    params_lgbm,
    scoring='f1',
    cv=3,   # количество блоков для кросс-валидации
    verbose=3  # видеть время
)

lgbm_gscv.fit(X_train, y_train)

print(f'Лучша оценка логистической регрессии: {lgbm_gscv.best_score_}')

print(f'Лучшие гиперпараметры логистической регрессии: {lgbm_gscv.best_params_}')

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV 1/3] END lgbmclassifier__max_depth=20, lgbmclassifier__n_estimators=300; total time= 1.5min
[CV 2/3] END lgbmclassifier__max_depth=20, lgbmclassifier__n_estimators=300; total time= 1.5min
[CV 3/3] END lgbmclassifier__max_depth=20, lgbmclassifier__n_estimators=300; total time= 1.6min
Лучша оценка логистической регрессии: 0.7314689877996535
Лучшие гиперпараметры логистической регрессии: {'lgbmclassifier__max_depth': 20, 'lgbmclassifier__n_estimators': 300}
CPU times: user 6min 55s, sys: 0 ns, total: 6min 55s
Wall time: 6min 58s


**Лучший результат показала модель LogisticRegression, её и проверим на тесте**

In [27]:
%%time
lr_predict = lr_gscv.predict(X_test)
f1_test = f1_score(y_test, lr_predict)
f1_test

CPU times: user 234 ms, sys: 0 ns, total: 234 ms
Wall time: 236 ms


0.7576036866359448

## Выводы

Проанализирован датасет с токсичными комментариями:
пропусков и дублей не обнаружено, выявлен дисбаланс классов, проведена очистка и лемматизация текста.

Данные разделены на две выборки с учетом равномерного распределения количества классов. Обучены три модели на предмет классификации комментариев на позитивные и негативные (0 или 1) с использованием кросс-валидации и поиском по сетке гиперпараметров.

На основе наибольшей оценки f1 выбрана модель логистической регрессии, и она проверена на тестовых данных. Получен f1>0,75